In [1]:
import xarray as xr
#zKuXR=xr.DataArray(np.array(zKuL),dims=['time','range'])
#zKu_trueXR=xr.DataArray(np.array(zKu_trueL),dims=['time','range'])
#piaKuXR=xr.DataArray(np.array(piaKuL),dims=['time'])
#dmXR=xr.DataArray(np.array(dm1DL),dims=['time','range'])
#rainRateXR=xr.DataArray(np.array(rainRate1DL),dims=['time','range'])
#attKuTopXR=xr.DataArray(np.array(attKuTopL),dims=['time'])
#ds=xr.Dataset({'zKu':zKuXR,'zKu_true':zKu_trueXR,'piaKu':piaKuXR,'dm':dmXR,'rainRate':rainRateXR,'attKuTop':attKuTopXR})
#compLev=5
#encoding={var : {'complevel': compLev} for var in ds.data_vars}
#ds.to_netcdf('simulated_radar_rain.nc',encoding=encoding)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import netCDF4 as nc
with nc.Dataset('simulated_radar_rain.nc') as ds:
    zKu=ds['zKu'][:]
    zKu_true=ds['zKu_true'][:]
    piaKu=ds['piaKu'][:]
    dm=ds['dm'][:]
    rainRate=ds['rainRate'][:]
    attKuTop=ds['attKuTop'][:]
    


FileNotFoundError: [Errno 2] No such file or directory: b'simulated_radar_rain.nc'

In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet1D(nn.Module):
    def __init__(self, in_channels=1, out_channels=2, npca=3, nin_channels=32):
        super(UNet1D, self).__init__()

        # Encoder
        self.conv1 = nn.Conv1d(in_channels, nin_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(nin_channels, nin_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(nin_channels, nin_channels*2, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(nin_channels*2, nin_channels*2, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.out_channels = out_channels
        self.npca = npca

        # Bottleneck
        self.bottleneck_conv1 = nn.Conv1d(nin_channels*2, nin_channels*2, kernel_size=3, padding=1)
        self.bottleneck_conv2 = nn.Conv1d(nin_channels*2, nin_channels*2, kernel_size=3, padding=1)

        # Decoder
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.conv5 = nn.Conv1d(4*nin_channels, 2*nin_channels, kernel_size=3, padding=1)
        self.conv6 = nn.Conv1d(2*nin_channels, nin_channels, kernel_size=3, padding=1)

        self.conv7 = nn.Conv1d(2*nin_channels, nin_channels, kernel_size=3, padding=1)
        self.conv8 = nn.Conv1d(nin_channels, out_channels*(1+npca), kernel_size=3, padding=1)

    def forward(self, x):
        # Encoder
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        x_pool1 = self.pool(x2)
        x3 = F.relu(self.conv3(x_pool1))
        x4 = F.relu(self.conv4(x3))
        x_pool = self.pool(x4)
        #print(x_pool.shape)
        # Bottleneck
        x_bottleneck = F.relu(self.bottleneck_conv1(x_pool))
        x_bottleneck = F.relu(self.bottleneck_conv2(x_bottleneck))
        # Decoder
        x_upsample1 = self.upsample(x_bottleneck)
        x_concat = torch.cat([x_upsample1, x3], dim=1)
        x5 = F.relu(self.conv5(x_concat))
        x6 = F.relu(self.conv6(x5))
        up_sample2=self.upsample(x6)
        x_concat2 = torch.cat([up_sample2, x2], dim=1)
        x7 = F.relu(self.conv7(x_concat2))
        x8 = (self.conv8(x7))
        x8_mu =  x8[:,:self.out_channels,:]
        x8_pca = x8[:,self.out_channels:,:]
        return x8_mu, x8_pca
# Example usage
unet_model = UNet1D(in_channels=1, nin_channels=32, npca=3, out_channels=2)
input_tensor = torch.randn(1, 1, 32)
output_tensor, output_pca = unet_model(input_tensor) 


In [75]:
print(output_tensor.shape)
print(output_pca.shape)
nz=32
print((output_tensor.view(-1,2*nz)).shape)

torch.Size([1, 2, 32])
torch.Size([1, 6, 32])
torch.Size([1, 64])


In [82]:
pca_v=torch.randn(1,1,3,2*nz)
def compute_cov(pca_vectors):
    cov = torch.matmul(pca_vectors.transpose(-2,-1), pca_vectors) 
    return cov

cov = compute_cov(pca_v)
print(cov.shape)

torch.Size([1, 1, 64, 64])


In [107]:
import torch.distributions as D
mu=torch.randn(2,1,64)
y=torch.randn(2,64)
pca_v=torch.randn(2,1,3,64)
def pca_mdn_loss(pi, mu, pca_vectors, y):
    batch_size, num_mixtures, output_dim = mu.size()
    y = y.unsqueeze(1).expand_as(mu)  # shape: [batch_size, num_mixtures, output_dim]
    
    # Compute covariance matrices from PCA vectors
    covariance = compute_cov(pca_vectors)
    
    # Calculate the log likelihoods for each mixture component
    mask = torch.eye(covariance.size(-1), device=covariance.device)  # Identity matrix
  # Expand the mask to match the tensor dimensions
    mask = mask.unsqueeze(0).unsqueeze(0)
    covariance = covariance + mask*0.1  # Add a small value to the diagonal for stability
    mvn = D.MultivariateNormal(loc=mu, covariance_matrix=covariance)
    log_probs = mvn.log_prob(y)
    
    # Combine log_probs with mixture weights
    log_probs = log_probs + torch.log(pi)
    
    # Log-sum-exp to combine mixture components
    log_sum_exp = torch.logsumexp(log_probs, dim=1)
    
    return -log_sum_exp.mean()
pi=torch.ones(2,1)
#mvn=D.MultivariateNormal(loc=mu, covariance_matrix=cov)
loss=pca_mdn_loss(pi, mu, pca_v, y)
print(loss)


tensor(631.7668)


In [98]:
print(cov.shape)
pca_v=torch.randn(10,1,3,2*nz)
cov = compute_cov(pca_v)
mask = torch.eye(cov.size(-1), device=cov.device)
mask = mask.unsqueeze(0).unsqueeze(0)
cov = cov + mask*0.1
print(mask.shape)
cov_numpy=cov.detach().numpy()
print(cov_numpy.shape)
print(np.linalg.eigvals(cov_numpy[0,0,:,:]))

torch.Size([10, 1, 64, 64])
torch.Size([1, 1, 64, 64])
(10, 1, 64, 64)
[38.78825    66.21794    70.00569     0.10000119  0.09999882  0.09999897
  0.10000084  0.09999916  0.0999992   0.10000077  0.09999928  0.1000007
  0.10000065  0.09999936  0.10000056  0.09999944  0.09999945  0.10000049
  0.09999951  0.10000043  0.09999958  0.1000004   0.09999961  0.09999965
  0.09999967  0.09999971  0.10000034  0.10000032  0.10000031  0.09999977
  0.10000028  0.10000026  0.09999982  0.10000023  0.10000022  0.10000021
  0.09999983  0.09999985  0.10000018  0.09999985  0.10000017  0.09999987
  0.09999987  0.10000014  0.0999999   0.0999999   0.10000013  0.10000011
  0.1000001   0.1000001   0.10000008  0.10000007  0.09999993  0.09999993
  0.10000005  0.09999996  0.09999996  0.09999997  0.10000004  0.10000003
  0.09999999  0.1         0.10000002  0.10000002]


In [108]:
print(output_tensor.shape)
zKum=zKu.mean(axis=0)
zKus=zKu.std(axis=0)
zKu_scaled=(zKu-zKum)/zKus
logRain=np.log(rainRate+1)
rainRatem=logRain.mean(axis=0)
rainRates=logRain.std(axis=0)
rainRate_scaled=(logRain-rainRatem)/rainRates
dm_m=dm.mean(axis=0)
dm_s=dm.std(axis=0)
dm_scaled=(dm-dm_m)/dm_s
from sklearn.model_selection import train_test_split
y=np.concatenate([rainRate_scaled[:,np.newaxis,:],dm_scaled[:,np.newaxis,:]],axis=1)
y=np.concatenate([rainRate_scaled[:,:],dm_scaled[:,:]],axis=1)
X_train, X_test, y_train, y_test = train_test_split(zKu_scaled, y, test_size=0.33, random_state=42)

torch.Size([1, 2, 32])


In [11]:
import torch.optim as optim

In [110]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
nt2=X_train.shape[0]//2
train_dataset =TensorDataset(torch.tensor(X_train[:nt2,np.newaxis,:][::10],dtype=torch.float32),torch.tensor(y_train[:nt2][::10],dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
print((train_dataset.tensors[1].shape),nt2,X_train[:nt2,np.newaxis,:].shape)
print(y_train[:nt2].shape)



torch.Size([8375, 64]) 83750 (83750, 1, 32)
(83750, 64)


In [111]:
train_dataset2 =TensorDataset(torch.tensor(X_train[nt2:,np.newaxis,:][::10],dtype=torch.float32),torch.tensor(y_train[nt2:][::10],dtype=torch.float32))
train_loader2 = DataLoader(train_dataset2, batch_size=32, shuffle=True)


In [32]:

unet_model1 = UNet1D(in_channels=1, nin_channels=32, out_channels=2)
criterion_mse=nn.MSELoss(reduction='mean')
optimizer = optim.Adam(unet_model1.parameters(), lr=0.001)
n_epochs = 5
for epoch in range(1, n_epochs + 1):
    train_loss = 0.0
    ibatch=0
    for x_batch, y_batch in train_loader:
        #print(x_batch.shape,y_batch.shape)
        optimizer.zero_grad()
        y=unet_model1(x_batch)  
        loss=criterion_mse(y, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        ibatch+=1
    train_loss = train_loss / ibatch
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

Epoch: 1 	Training Loss: 0.570751
Epoch: 2 	Training Loss: 0.469074
Epoch: 3 	Training Loss: 0.444670
Epoch: 4 	Training Loss: 0.432290
Epoch: 5 	Training Loss: 0.421365


In [28]:

unet_model2 = UNet1D(in_channels=1, nin_channels=32, out_channels=2)
criterion_mse=nn.MSELoss(reduction='mean')
optimizer = optim.Adam(unet_model2.parameters(), lr=0.001)
n_epochs = 5
for epoch in range(1, n_epochs + 1):
    train_loss = 0.0
    ibatch=0
    for x_batch, y_batch in train_loader2:
        #print(x_batch.shape,y_batch.shape)
        optimizer.zero_grad()
        y=unet_model2(x_batch)  
        loss=criterion_mse(y, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        ibatch+=1
    train_loss = train_loss / ibatch
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

Epoch: 1 	Training Loss: 0.428620
Epoch: 2 	Training Loss: 0.386310
Epoch: 3 	Training Loss: 0.377654
Epoch: 4 	Training Loss: 0.372665
Epoch: 5 	Training Loss: 0.369760


In [33]:
yout1=unet_model1(torch.tensor(X_test[:,np.newaxis,:],dtype=torch.float32))
print(np.corrcoef(yout1.detach().numpy()[:,0,-1],y_test[:,0,-1]))

[[1.        0.5668979]
 [0.5668979 1.       ]]


In [34]:
#yout2=unet_model2(torch.tensor(X_test[:,np.newaxis,:],dtype=torch.float32))
print(np.corrcoef(yout2.detach().numpy()[:,0,-1],yout1.detach().numpy()[:,0,-1]))#y_test[:,0,-1]))

[[1.         0.91743134]
 [0.91743134 1.        ]]


In [62]:
from torch.distributions.multivariate_normal import MultivariateNormal
a=np.array([[1.0, 1.0], [0.5, 0.0]])
a_tensor=torch.tensor(a,dtype=torch.float32)
m_tensor=torch.tensor([0.0, 0.0],dtype=torch.float32)
print(a_tensor)
print(m_tensor)
#m = MultivariateNormal(loc=m_tensor, scale_tril=a_tensor)
loc = torch.zeros(3)
scale = torch.ones(3)
print(torch.diag(scale))
cholesky=torch.diag(scale)
print(cholesky.dtype)
cholesky[1,2]=0.5
cov_a=torch.mm(cholesky,cholesky.t())
print(cholesky.dtype)
mvn = MultivariateNormal(loc, covariance_matrix=cov_a)

tensor([[1.0000, 1.0000],
        [0.5000, 0.0000]])
tensor([0., 0.])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
torch.float32
torch.float32


In [60]:
torch.tril_indices(3, 3, offset=0, dtype=torch.int64, device=None)
x1=np.array([1,0])
np.dot(x1[:,np.newaxis],x1[:,np.newaxis].T)

array([[1, 0],
       [0, 0]])